# Выбор локации для скважины

В добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
import numpy as np
import matplotlib.pyplot as plt
import math
from numpy.random import RandomState
from scipy import stats
from sklearn.utils import shuffle

### Загрузка

In [2]:
# Пути к файлам
files = ['geo_data_0.csv', 'geo_data_1.csv', 'geo_data_2.csv']
paths = ['datasets\\', 'C:\\Downloads\\']

# Словарь для хранения загруженных данных
data = {}

for file in files:
    for path in paths:
        try:
            # Попытка загрузить данные
            file_path = os.path.join(path, file)
            data[file] = pd.read_csv(file_path)
            print(f"Файл {file} успешно загружен из {path}")
            break
        except FileNotFoundError:
            print(f"Файл {file} не найден в {path}. Попытка загрузить из другого пути.")
        except Exception as e:
            print(f"Произошла ошибка при загрузке файла {file} из {path}: {e}")



Файл geo_data_0.csv успешно загружен из datasets\
Файл geo_data_1.csv успешно загружен из datasets\
Файл geo_data_2.csv успешно загружен из datasets\


In [3]:
# Вывод информации о данных
for file, df in data.items():
    print(f"\nИнформация о {file}:")
    print(df.info())
    print(f"Первые 5 строк {file}:")
    display(df.head())
    print(f"Статистический анализ {file}:")
    display(df.describe())


Информация о geo_data_0.csv:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
Первые 5 строк geo_data_0.csv:


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


Статистический анализ geo_data_0.csv:


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347



Информация о geo_data_1.csv:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
Первые 5 строк geo_data_1.csv:


...

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408



Информация о geo_data_2.csv:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
Первые 5 строк geo_data_2.csv:


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


Статистический анализ geo_data_2.csv:


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


In [4]:
# Создание отдельной переменной для каждого файла для удобства
geo_data_0 = data['geo_data_0.csv']
geo_data_1 = data['geo_data_1.csv']
geo_data_2 = data['geo_data_2.csv']

### Подготовка данных

In [5]:
# Определение признаков и целевого значения
features = ['f0', 'f1', 'f2']
target = 'product'

# Разделение данных для geo_data_0
X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(geo_data_0[features], geo_data_0[target], test_size=0.25, random_state=13)

# Разделение данных для geo_data_1
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(geo_data_1[features], geo_data_1[target], test_size=0.25, random_state=13)

# Разделение данных для geo_data_2
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(geo_data_2[features], geo_data_2[target], test_size=0.25, random_state=13)

In [6]:
# Создание списка экземпляров RobustScaler
scalers = [RobustScaler() for _ in range(3)]

# Масштабирование тренировочных данных и тестовых данных
X_train_scaled = [scaler.fit_transform(X_train) for scaler, X_train in zip(scalers, [X_train_0, X_train_1, X_train_2])]
X_test_scaled = [scaler.transform(X_test) for scaler, X_test in zip(scalers, [X_test_0, X_test_1, X_test_2])]

Вот последовательность действий в представленном коде:

1. **Импорт необходимых библиотек**: Импортируются все необходимые библиотеки для обработки данных, моделирования и визуализации.

2. **Определение путей к файлам**: Задаются имена файлов и пути к ним.

3. **Загрузка данных**: Данные загружаются из указанных файлов. Если файл не найден по одному пути, происходит попытка загрузить его из другого пути.

4. **Вывод информации о данных**: Для каждого загруженного файла выводится информация о данных, первые 5 строк и статистический анализ.

5. **Определение признаков и целевого значения**: Определяются признаки (`f0`, `f1`, `f2`) и целевое значение (`product`).

6. **Разделение данных на обучающую и тестовую выборки**: Данные каждого файла разделяются на обучающую и тестовую выборки с использованием функции `train_test_split`.

7. **Масштабирование данных**: Данные масштабируются с использованием `RobustScaler`, который устойчив к выбросам. Масштабирование применяется отдельно к обучающим и тестовым данным каждого файла.

После выполнения этих шагов данные готовы к дальнейшему анализу и моделированию.

## Обучение и проверка модели

In [7]:
# Создание списка экземпляров LinearRegression
models = [LinearRegression() for _ in range(3)]

# Обучение моделей на масштабированных тренировочных данных
[model.fit(X_train, y_train) for model, X_train, y_train in zip(models, X_train_scaled, [y_train_0, y_train_1, y_train_2])]

# Предсказание на масштабированных тестовых данных
y_pred = [model.predict(X_test) for model, X_test in zip(models, X_test_scaled)]


In [8]:
def process_data(X_test, y_test, y_pred):
    # Объединение тестовых признаков и целевых переменных
    test_data = pd.concat([X_test, y_test], axis=1)
    # Добавление предсказаний в качестве нового столбца
    test_data['y_pred'] = y_pred
    return test_data

# Применение функции к данным
test_data_0 = process_data(X_test_0, y_test_0, y_pred[0])
test_data_1 = process_data(X_test_1, y_test_1, y_pred[1])
test_data_2 = process_data(X_test_2, y_test_2, y_pred[2])

# Вывод первых 5 строк тестового набора для каждого набора данных
# а также среднего запаса предсказанного сырья, RMSE и MAE для каждого набора данных
for i, data in enumerate([test_data_0, test_data_1, test_data_2]):
    print(f"\nПервые 5 строк тестового набора для geo_data_{i}:")
    display(data.head())
    avg_pred_res = np.mean(data['y_pred'])
    rmse_ = np.sqrt(mean_squared_error(data['product'], data['y_pred']))
    mae = mean_absolute_error(data['product'], data['y_pred'])
    print(f'\nСредний запас предсказанного сырья для тестового набора {i}: {avg_pred_res}')
    print(f'RMSE для тестового набора {i}: {rmse_}')
    print(f'MAE для тестового набора {i}: {mae}')




Первые 5 строк тестового набора для geo_data_0:


,f0,f1,f2,product,y_pred
72031,0.344571,-0.272941,-2.099642,170.053522,68.927415
27978,1.652531,-0.206616,1.868818,61.453092,99.176667
55639,-0.253885,0.993549,1.584251,145.743331,73.078440
51955,1.085273,0.155305,-0.680978,42.100082,75.076824
52145,0.017405,0.081738,-1.458572,99.160596,66.902659



Средний запас предсказанного сырья для тестового набора 0: 92.5943198620576
RMSE для тестового набора 0: 37.694547354090695
MAE для тестового набора 0: 31.008550751783346

Первые 5 строк тестового набора для geo_data_1:


,f0,f1,f2,product,y_pred
72031,2.404544,1.392635,3.003099,80.859783,82.215551
27978,7.863461,-3.074607,4.994210,134.766305,135.186645
55639,-10.829960,-4.771709,3.006065,84.038886,84.348269
51955,10.964350,-3.753278,4.992659,134.766305,134.710099
52145,16.457302,-7.863488,4.000320,107.813044,107.257408



Средний запас предсказанного сырья для тестового набора 1: 69.02949151591964
RMSE для тестового набора 1: 0.8937965421900188
MAE для тестового набора 1: 0.7199001055406402

Первые 5 строк тестового набора для geo_data_2:


,f0,f1,f2,product,y_pred
72031,-0.370020,0.667530,-4.538057,76.447828,54.556699
27978,1.525227,-4.025469,1.634140,60.054617,90.189453
55639,0.881421,4.007687,10.713461,160.069409,141.822812
51955,2.229994,-1.056556,-0.747277,93.090521,76.183577
52145,2.442010,1.116135,8.983982,133.124375,131.928489



Средний запас предсказанного сырья для тестового набора 2: 94.95620175786401
RMSE для тестового набора 2: 40.021834799355375
MAE для тестового набора 2: 32.755053018732


Код выполняет следующие действия:

1. **Создание моделей**: Создаются три экземпляра модели линейной регрессии, по одному для каждого набора данных.

2. **Обучение моделей**: Модели обучаются на масштабированных тренировочных данных.

3. **Предсказание на тестовых данных**: С помощью обученных моделей делаются предсказания на масштабированных тестовых данных.

4. **Обработка данных**: Создается функция `process_data`, которая объединяет тестовые признаки и целевые переменные, а затем добавляет предсказания в качестве нового столбца. Эта функция применяется к данным.

5. **Анализ результатов**: Выводятся первые 5 строк тестового набора для каждого набора данных, а также средний запас предсказанного сырья, RMSE и MAE для каждого набора данных.

Результаты показывают, что модели имеют различное качество предсказаний для разных наборов данных. Например, модель для `geo_data_1` имеет наименьшие значения RMSE и MAE, что указывает на лучшую точность предсказаний по сравнению с другими моделями. С другой стороны, модель для `geo_data_0` имеет наибольшее значение RMSE, что указывает на наибольшую ошибку предсказаний.

Это может быть связано с различиями в распределении данных в разных наборах данных.

Также стоит отметить, что средний запас предсказанного сырья варьируется между наборами данных. Это может быть важным фактором при принятии решений в контексте бизнес-задачи. 


## Подготовка к расчёту прибыли

In [9]:
# Ключевые значения для расчётов
СРЕДНИЙ_ЗАПАС_ПРЕДСКАЗАННОГО_СЫРЬЯ_0 = np.mean(test_data_0['y_pred'])
СРЕДНИЙ_ЗАПАС_ПРЕДСКАЗАННОГО_СЫРЬЯ_1 = np.mean(test_data_1['y_pred'])
СРЕДНИЙ_ЗАПАС_ПРЕДСКАЗАННОГО_СЫРЬЯ_2 = np.mean(test_data_2['y_pred'])
БЮДЖЕТ_НА_РАЗРАБОТКУ = 10000000000  # в рублях
ДОХОД_С_ЕДИНИЦЫ_ПРОДУКТА = 450000  # в рублях, поскольку объём указан в тысячах баррелей
КОЛИЧЕСТВО_СКВАЖИН = 200
БЕЗУБ_ОБ = БЮДЖЕТ_НА_РАЗРАБОТКУ / (ДОХОД_С_ЕДИНИЦЫ_ПРОДУКТА * 200)


In [10]:
def check_volume(break_even_volume, mean_volume, region):
    mean_volume = math.ceil(mean_volume)
    if break_even_volume > mean_volume:
        result = f'В регионе {region} средний запас предсказанного сырья недостаточен для безубыточной разработки'
    else:
        result = f'В регионе {region} средний запас предсказанного сырья достаточен для безубыточной разработки'
    return result + f'. Средний объем в регионе: {mean_volume}'

# Выводим достаточный объём на экран
print('Достаточный объём сырья для безубыточной разработки новой скважины:', БЕЗУБ_ОБ)

# Сравниваем полученный объём сырья со средним запасом в каждом регионе
print(check_volume(БЕЗУБ_ОБ, СРЕДНИЙ_ЗАПАС_ПРЕДСКАЗАННОГО_СЫРЬЯ_0, 0))
print(check_volume(БЕЗУБ_ОБ, СРЕДНИЙ_ЗАПАС_ПРЕДСКАЗАННОГО_СЫРЬЯ_1, 1))
print(check_volume(БЕЗУБ_ОБ, СРЕДНИЙ_ЗАПАС_ПРЕДСКАЗАННОГО_СЫРЬЯ_2, 2))


Достаточный объём сырья для безубыточной разработки новой скважины: 111.11111111111111
В регионе 0 средний запас предсказанного сырья недостаточен для безубыточной разработки. Средний объем в регионе: 93
В регионе 1 средний запас предсказанного сырья недостаточен для безубыточной разработки. Средний объем в регионе: 70
В регионе 2 средний запас предсказанного сырья недостаточен для безубыточной разработки. Средний объем в регионе: 95


In [11]:
# Сортируем данные по предсказанному объему сырья и выбираем 200 лучших скважин
top_200_wells_0 = test_data_0.sort_values('y_pred', ascending=False).head(200)
top_200_wells_1 = test_data_1.sort_values('y_pred', ascending=False).head(200)
top_200_wells_2 = test_data_2.sort_values('y_pred', ascending=False).head(200)

# Считаем средний запас предсказанного сырья для 200 лучших скважин
mean_top_200_wells_0 = np.mean(top_200_wells_0['y_pred'])
mean_top_200_wells_1 = np.mean(top_200_wells_1['y_pred'])
mean_top_200_wells_2 = np.mean(top_200_wells_2['y_pred'])

# Сравниваем полученный объём сырья со средним запасом в каждом регионе
print(check_volume(БЕЗУБ_ОБ, mean_top_200_wells_0, 0))
print(check_volume(БЕЗУБ_ОБ, mean_top_200_wells_1, 1))
print(check_volume(БЕЗУБ_ОБ, mean_top_200_wells_2, 2))


В регионе 0 средний запас предсказанного сырья достаточен для безубыточной разработки. Средний объем в регионе: 156
В регионе 1 средний запас предсказанного сырья достаточен для безубыточной разработки. Средний объем в регионе: 139
В регионе 2 средний запас предсказанного сырья достаточен для безубыточной разработки. Средний объем в регионе: 149


Исходные расчеты показали, что для безубыточной разработки новой скважины необходим объем сырья в **111.11** тысяч баррелей. Однако, средний запас предсказанного сырья во всех трех регионах был ниже этого значения:

- В регионе 0 средний запас составляет **93** тысячи баррелей.
- В регионе 1 средний запас составляет **70** тысяч баррелей.
- В регионе 2 средний запас составляет **95** тысяч баррелей.

Это означает, что с точки зрения безубыточности, ни один из регионов не достигает необходимого уровня запасов сырья.

Однако, когда мы рассмотрели средний запас предсказанного сырья для 200 лучших скважин, ситуация изменилась:

- В регионе 0 средний запас составляет **156** тысяч баррелей.
- В регионе 1 средний запас составляет **139** тысяч баррелей.
- В регионе 2 средний запас составляет **149** тысяч баррелей.

Таким образом, если мы сосредоточимся на 200 лучших скважинах, все три региона достигают необходимого уровня запасов сырья для безубыточной разработки. Это подчеркивает важность выбора правильных скважин для разработки. <br>

## Расчёт прибыли и рисков 

In [12]:
def analyze_regions(data_0, data_1, data_2, n_iterations, exploration_wells, best_wells):
    state = RandomState(13)

    def profit_calculation(data, exploration_wells, best_wells):
        # Перемешиваем данные
        data = shuffle(data, random_state=13)
        # Выбираем скважины для разведки
        exploration_data = data.head(exploration_wells)
        # Выбираем лучшие скважины с максимальными значениями предсказаний
        selected_wells = exploration_data.sort_values(by='y_pred', ascending=False).head(best_wells)
        # Просуммируем целевое значение объёма сырья, соответствующее этим предсказаниям
        total_volume = selected_wells['product'].sum()
        # Рассчитываем прибыль для полученного объёма сырья
        profit = total_volume * ДОХОД_С_ЕДИНИЦЫ_ПРОДУКТА - БЮДЖЕТ_НА_РАЗРАБОТКУ
        return profit

    def bootstrap(data, n_iterations, exploration_wells, best_wells):
        values = []
        for i in range(n_iterations):
            subsample = data.sample(n=exploration_wells, replace=True, random_state=state)
            profit = profit_calculation(subsample, exploration_wells, best_wells)
            values.append(profit)
        values = pd.Series(values)
        mean_profit = values.mean()
        lower = values.quantile(0.025)
        upper = values.quantile(0.975)
        risk = (values < 0).mean()
        return mean_profit, lower, upper, risk

    results = pd.DataFrame(columns=['Регион', 'Средняя прибыль', '95%-й доверительный интервал (нижняя граница)', '95%-й доверительный интервал (верхняя граница)', 'Риск убытков'])
    for i, data in enumerate([data_0, data_1, data_2]):
        mean_profit, lower, upper, risk = bootstrap(data, n_iterations, exploration_wells, best_wells)
        results.loc[i] = [f'Регион {i}', round(mean_profit), round(lower), round(upper), round(risk, 2)]

    safe_regions = results[results['Риск убытков'] < 0.025]
    best_region = safe_regions[safe_regions['Средняя прибыль'] == safe_regions['Средняя прибыль'].max()]['Регион'].values[0]

    display(results)
    print(f'Лучший регион для разработки скважин: {best_region}')

analyze_regions(test_data_0, test_data_1, test_data_2, 1000, 500, 200)





,Регион,Средняя прибыль,95%-й доверительный интервал (нижняя граница),95%-й доверительный интервал (верхняя граница),Риск убытков
0,Регион 0,441880880,-70804342,946595002,0.05
1,Регион 1,465233763,76043311,843830277,0.01
2,Регион 2,375040087,-133807722,897746247,0.08


Лучший регион для разработки скважин: Регион 1


На основании проведенного анализа, **Регион 1** является наиболее подходящим для разработки скважин. 

Вот несколько ключевых моментов, которые обосновывают этот выбор:

1. **Средняя прибыль**: Регион 1 показывает наибольшую среднюю прибыль среди всех регионов, составляющую 465233763.

2. **95%-й доверительный интервал**: Доверительный интервал для Региона 1 составляет от 76043311 до 843830277, что указывает на стабильность и надежность прогнозируемой прибыли.

3. **Риск убытков**: Регион 1 имеет наименьший риск убытков среди всех регионов, составляющий всего 0.01 или 1%.

Таким образом, учитывая все эти факторы, Регион 1 является наиболее предпочтительным для разработки скважин.